In [1]:
#数据， 模型， 优化器， 损失函数， 训练

In [2]:
import sys
sys.path.insert(0, '../..')
%matplotlib inline
import d2l
import torch, torchvision
from torch.utils.data import DataLoader

In [70]:
#使用d2l的函数来加载数据，不用前一节的一步步自己实现了
batch_size=256
train_dataloader, test_dataloader = d2l.load_data_fashion_mnist(batch_size=batch_size)

In [84]:
#输入是256*(1*28*28), 输出是256*10     初始化参数
num_inputs = 28*28
num_outputs = 10
W = torch.Tensor(num_inputs, num_outputs).normal_()
b = torch.Tensor(num_outputs).normal_(std=0.01)

In [85]:
W.requires_grad_(True)
b.requires_grad_(True)

tensor([-0.0006, -0.0127, -0.0055, -0.0036,  0.0042, -0.0031, -0.0174,  0.0129,
         0.0033,  0.0042], requires_grad=True)

In [86]:
b.requires_grad

True

In [87]:
#softmax fun
def softmax(y):
    exp_y = torch.exp(y)
    deno = torch.sum(exp_y, dim=1, keepdim=True)
    return exp_y / deno  #here will use broadcast to full deno

In [88]:
def softmax_regression(X):
    y = torch.mm(torch.reshape(X, (-1, num_inputs)), W) + b
    p = softmax(y)
    return p


In [89]:
#测试一下 softmax_regression
for X, y in train_dataloader:
    p = softmax_regression(X)
    print(p.shape)
    print(y)
    break

torch.Size([256, 10])
tensor([9, 7, 8, 8, 2, 5, 9, 0, 7, 2, 9, 5, 8, 2, 5, 3, 2, 1, 4, 3, 6, 1, 2, 1,
        4, 8, 3, 1, 3, 3, 3, 6, 0, 6, 1, 0, 9, 5, 0, 8, 5, 0, 8, 0, 4, 3, 1, 4,
        7, 1, 7, 9, 3, 1, 9, 2, 7, 2, 8, 5, 3, 9, 8, 7, 0, 6, 4, 6, 1, 2, 1, 4,
        7, 2, 5, 0, 5, 2, 7, 9, 0, 0, 1, 5, 6, 0, 4, 0, 1, 0, 2, 3, 6, 8, 8, 2,
        2, 9, 9, 8, 0, 1, 6, 3, 9, 4, 5, 8, 3, 3, 1, 2, 0, 6, 6, 2, 7, 8, 6, 8,
        0, 4, 2, 4, 2, 4, 3, 4, 3, 6, 6, 9, 2, 7, 3, 2, 1, 6, 4, 4, 8, 4, 6, 3,
        4, 8, 3, 5, 7, 6, 7, 9, 5, 1, 6, 0, 1, 0, 7, 4, 7, 0, 2, 4, 1, 2, 7, 0,
        9, 0, 2, 8, 4, 7, 9, 3, 6, 8, 6, 8, 2, 3, 3, 7, 7, 3, 5, 5, 8, 5, 6, 7,
        0, 3, 0, 5, 1, 6, 7, 8, 6, 9, 6, 3, 3, 4, 5, 0, 8, 8, 5, 4, 8, 0, 0, 0,
        4, 0, 1, 2, 5, 4, 7, 4, 9, 8, 3, 9, 2, 9, 9, 6, 9, 3, 9, 4, 1, 1, 5, 7,
        6, 4, 4, 6, 9, 7, 7, 0, 7, 8, 6, 6, 3, 1, 4, 7])


In [90]:
#模拟一下三分类的 p 和 label
label = torch.LongTensor([0, 2]) #两个样本分别是第0类和第2类
p = torch.Tensor([[0.1, 0.3, 0.6], [0.3, 0.2, 0.5]])
#cross_entropy_loss = y*log(p) = log(p_i[label_i])  : loss1->log(0.1)    loss2->log(0.5) 
print(label.unsqueeze(dim=1))
print(torch.gather(p, 1, label.unsqueeze(dim=1)))

#loss
def cross_entropy_loss(p, label):
    return -torch.gather(p, 1, label.unsqueeze(dim=1)).log()

tensor([[0],
        [2]])
tensor([[0.1000],
        [0.5000]])


In [91]:
def get_acc(data_iter, softmax_regression):
    acc_sum = 0
    count = 0
    for X, y in data_iter:
        p = softmax_regression(X)
        acc_sum += (p.argmax(dim=1) == y).sum().item()
        count += y.size()[0]
    return acc_sum / count

In [95]:
#训练
lr = 0.1
epochs = 20
for epoch in range(epochs):
    for X, y in train_dataloader:
        p = softmax_regression(X)
        loss = cross_entropy_loss(p, y).mean()
        
        loss.backward()
        
        W.data.sub_(lr*W.grad/batch_size)
        b.data.sub_(lr*b.grad/batch_size)
        
        W.grad.data.zero_()
        b.grad.data.zero_()
    print('epoch {}, acc is {}'.format(epoch+1, get_acc(train_dataloader, softmax_regression)))
        
        

epoch 1, acc is 0.14741666666666667
epoch 2, acc is 0.1524
epoch 3, acc is 0.15611666666666665
epoch 4, acc is 0.16045
epoch 5, acc is 0.16586666666666666
epoch 6, acc is 0.17201666666666668
epoch 7, acc is 0.17875
epoch 8, acc is 0.18555
epoch 9, acc is 0.1926
epoch 10, acc is 0.19891666666666666
epoch 11, acc is 0.20541666666666666
epoch 12, acc is 0.21171666666666666
epoch 13, acc is 0.21791666666666668
epoch 14, acc is 0.22288333333333332
epoch 15, acc is 0.22871666666666668
epoch 16, acc is 0.23358333333333334
epoch 17, acc is 0.23828333333333335
epoch 18, acc is 0.24226666666666666
epoch 19, acc is 0.24646666666666667
epoch 20, acc is 0.2508666666666667
